# `bw_timex` Teaching Exercise - Time-explicit LCA of an Electric Vehicle vs Petrol Vehicle

In this exercise notebook you will compare the climate performance of a battery electric vehicle (BEV) and an internal combustion engine car (ICEC), in a time explicit way, using different decarbonisation scenarios. 

In order to have a fair comparison**, we will start from two existing ecoinvent activities (listed below) and will add the temporal information as needed. This notebook will guide you through the steps needed to build the model. But if you are you can either ask the instructors or have a look at the solution notebook (Note that this just provides one solution, and there are often multiple ways to get to the same result, so we encourage you to use the way that is most intuative to you!). 

At the end of this notebook there are a few bonus questions. 


> ** Warning: Any LCA is only every as good as the underlying data. the data on cars in Ecoinvent is really outdated (check the description/comment of the activities to get an idea about the data), so the results from this exercise should be interpreted with care, as should any LCA result be of course!

In [ ]:
import bw2data as bd

bd.projects.set_current("timex_teaching")

## Prospective background databases

In addition to the very optimistic PkBudg650 scenario used in the previous notebook, we'll additionally consider a 'national policies implemented' (NPi) as a business as usual (BAU) scenario here:

- REMIND-EU - SSP2-NPi       : BAU scenario with ~3˚ warming by 2100
- REMIND-EU - SSP2-PkBudg650 : Optimistic scenario in line with Paris 2015 with 1.5˚-1.9˚ warming by 2100

> See [the Premise documentation](https://premise.readthedocs.io/en/latest/introduction.html#default-iam-scenarios) for a brief overview of the IAM scenarios

Let's get easy handles for the databases for ease of use:

National Policies and Implemention (NPi) scenario:

In [ ]:
db_2020_NPi = bd.Database("ei_cutoff_3.10_remind-eu_SSP2-NPi_2020 2025-10-08")
db_2030_NPi = bd.Database("ei_cutoff_3.10_remind-eu_SSP2-NPi_2030 2025-10-08")
db_2040_NPi = bd.Database("ei_cutoff_3.10_remind-eu_SSP2-NPi_2040 2025-10-08")
db_2050_NPi = bd.Database("ei_cutoff_3.10_remind-eu_SSP2-NPi_2050 2025-10-08")

Peak-Budget 650 scenario:

In [ ]:
db_2020_PkBudg650 = bd.Database("ei_cutoff_3.10_remind-eu_SSP2-PkBudg650_2020 2025-10-08")
db_2030_PkBudg650 = bd.Database("ei_cutoff_3.10_remind-eu_SSP2-PkBudg650_2030 2025-10-08")
db_2040_PkBudg650 = bd.Database("ei_cutoff_3.10_remind-eu_SSP2-PkBudg650_2040 2025-10-08")
db_2050_PkBudg650 = bd.Database("ei_cutoff_3.10_remind-eu_SSP2-PkBudg650_2050 2025-10-08")

## Case study setup

In this exercise we will go about setting up the forground system in a slightly different way. In our case driving a(n electric) car is already represented in ecoinvent. So all we would need to do is to add the temporal distributions to the exchanges. 

In the current implementation of `bw_timex`, all activities containing temporal information on the exchanges need to 'live' in the foreground system, so that their inputs can get linked to different background activities.

Let's start the definition of our functional unit (FU): 

- Driving a passenger car for 200,000km (including production, EoL treatment, maintenance, etc), over a period of 15 years.

### Modeling the production system

As a first step of modelling the production system let's create a new foreground database:


In [ ]:
fg_db_name = "foreground_bev_vs_icec"

if fg_db_name in bd.databases:
    del bd.databases[fg_db_name] # to make sure we create the foreground from scratch

In [ ]:
foreground = bd.Database(fg_db_name)
foreground.register()

And let's define the FU relevant quantities: 

In [ ]:
LIFETIME_KM = 200000.  # lifetime of the vehicle in km
LIFETIME = 15  # lifetime of the vehicle in years
ANNUAL_KM = LIFETIME_KM / LIFETIME  # annual km

Now let's create two activities in the foreground that represent the functional units as defined above (production/driving/EoL treatment of a car for 200,000km over 15 years). We can call them LC_BEV and LC_ICEC for example (LC=life cyle)

Let's make a simplified overview of the activities at every step (example for the ICEC). So far we only have defined the lifecycle activity:
in the foreground. We will use different colours for the foreground and backgroudn processes. 

... DIY

Ecoinvent already contains activities that represent transport with a passenger car. So let's get the following two activities from the base database** for the BAU scenario and create a copy in our foreground database:

- Driving a gasoline car: "transport, passenger car, gasoline, Medium SUV, EURO-6"
- Driving a BEV: "transport, passenger car, gasoline, Medium SUV, EURO-6"

The idea is to first see how these activities are set up. It might help to draw your own system definition to create an overview of the exchnages and to which you want to add temporal information. The following steps guide you through the process. 

HINT: their are multiple locations of this activity in the premise database, choose one of your liking/or that makes sense for you.
HINT: copy to a different database by first copying the activity and then changing the database.

> ** We consider that as our base year and db_2020 as the 'base' database

The Reference product of these activities should be 1 km traveled by the resepctive vehicle, but let's check the reference product and its unit:

Have a look at what the inputs into such a car is and identify the relevant exchanges:

Let's make a simplified overview of these activities so far (example for the ICEC): 

... DIY

These 'transport activities' include both the car, which in itself contains both production and EoL treatment (check this!!), as well as the fuel/electricity, maintenance and wear etc. Now we want to model the production and EoL treatment of car and it's elements separately.

So, let's make a copy of these activities in the foreground so that we can add temporal distributions to the production and EoL.

At the same time we want to split the driving and production/EoL so set the input of the car into the driving activity to 0 to avoid double counting.

Have a look at the exchanges of the production/EoL activity and see that these indeed include both production and EoL.


Let's update our overview:

...DIY



### Adding the relevant exchanges and temporal information

Now it is time to add the relevant exchanges to the functional unit activities together with relevant temporal distributions. Be carefull to add the temporal distribution to poduction and EoL at the right exchnage, so that production happens before the use phase but EoL after the use phase. 

Tipp: for ease of use, or if you forgot how to define temporal distributions you can use the interactive_td_widget!

Tipp 2: Use the utility function add_temporal_distribution_to_exchange to add the temporal distribution to the EoL of the production/EoL activity of the cars. 


Let's first draw the system we want including where we want TD's


... DIY




Now define and add the temporal distributions to the relevant exchanges.

Some relevant imports:

In [ ]:
import numpy as np

from bw_temporalis import TemporalDistribution, easy_timedelta_distribution
from bw_timex.utils import add_temporal_distribution_to_exchange

Tipp: for ease of use, or if you forgot how to define temporal distributions you can use the interactive_td_widget!
You might have to run this cell twice for the widget to show up.

In [ ]:
from bw_timex.utils import interactive_td_widget

interactive_td_widget()

### Checking all went right

Now before calculating the time-explicit LCA, make sure that everything went right so far. As a check, the standard LCA score of the original transport activity with the corresponding amount should be equal the standard LCA of the modified life cycle activity created in the foreground!

In [ ]:
# import bw2calc for standard LCA calculations
import bw2calc as bc



## Time-explicit LCA with `bw_timex`

Now we can start using `bw_timex` to build the process timeline, build the time-explicit inventory and calculate the time explicit scores.

Start by creating a dictionary mapping the respective databases to the relevant timestamps. 
Then instantiate your timexLCA object, build the timeline, calculate the lci, and the LCIA for both options.


In [ ]:
# import datetime for defining database dates
from datetime import datetime



In [ ]:
# import TimexLCA for time-explicit LCA calculations
from bw_timex import TimexLCA

## Visualise Results and compare the different cars and scenarios

- Now find a good way to visualise the results. There are some simple plotting functions included such as plot_dynamic_inventory().

- Run the other scenario to see the differences for both cars. 

## BONUS options:

- Add temporal information further throughout the supply chain. To do this, move further background processes to the foreground and add TDs, e.g., for the steel that is needed for the vehicle production processes

- Use temporal distributions to model a fleet (of changing composition). (hint a for a fleet some cars will live longer, some shorter. This can be modelled using a distribution for the use phase and the EoL)
    - Now you can also look at different impacts (e.g. metal demand for example and see the metal demand per year needed to build a fleet)

- How is the comparison if we replace the petrol consumption of the car with biofuel (e.g. this activity:  'petrol production, 85% ethanol by volume from biomass' (kilogram, CH, None)). And then what about if you include biogenic carbon emissions by using for example this method: ('ecoinvent-3.10',
  'IPCC 2021',
  'climate change: biogenic, including SLCFs',
  'global warming potential (GWP20)')

- Use dynamic_lcia(use_disaggregated_lci=TRUE) to perform a contribution analysis on the background activities contributing to the impacts. (Note that this function can take a while), find a good way to visualize this